# Local Satisfaction data preparation  
This notebook shows the process of getting data from **wide format** (for each observation, as many columns as questions) to **long format** (a column for the question, and a column for the answer, for all the data)

In [88]:
import pandas as pd 
import numpy as np

## 1) Load data  
Load the original data from the google sheet

In [89]:
sheet_id = '1iXFCOE7iAhpajY9v2GjtZM21GDDTjWFRzeO8Q_InP7E'
sheet_name = 'simplified_data'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

original_data = pd.read_csv(url)
original_data['id'] = original_data.index
original_data.columns = original_data.columns.str.strip()
original_data.head()

,Satisfaction with life,Sense of belonging in community,Satisfaction with tourism,Satisfaction with tourism .1,Tourism,Jobs with tourism,Entrepreneurship with tourism,Local culture from tourism,Production of local productions with tourism,Views on policies on tourism,Satisfaction with the quality of basic educational services,Satisfaction with the quality of basic healh services,Satisfaction with access to recreation,Satisfaction with access to cultural activities,Satisfaction with safety,Sense of a healthy environment,Demographic,id
0,80.0,75.0,70.0,Increase,NaN,NaN,100.0,100.0,100.0,100.0,100.0,NaN,75.0,100.0,50.0,75.0,Central Okanagan,0
1,60.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,50.0,100.0,75.0,NaN,1
2,80.0,100.0,NaN,Increase,NaN,NaN,100.0,100.0,100.0,100.0,100.0,NaN,100.0,0.0,100.0,75.0,Central Okanagan,2
3,70.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,75.0,75.0,50.0,NaN,3
4,70.0,100.0,90.0,Increase,Nearby BC communities | All of BC | Ot...,British Columbia Visitors | Canadian V...,75.0,75.0,75.0,75.0,75.0,Yes,75.0,75.0,50.0,50.0,Central Okanagan,4


In [90]:
original_data.columns

Index(['Satisfaction with life', 'Sense of belonging in community',
       'Satisfaction with tourism', 'Satisfaction with tourism .1', 'Tourism',
       'Jobs with tourism', 'Entrepreneurship with tourism',
       'Local culture from tourism',
       'Production of local productions with tourism',
       'Views on policies on tourism',
       'Satisfaction with the quality of basic educational services',
       'Satisfaction with the quality of basic healh services',
       'Satisfaction with access to recreation',
       'Satisfaction with access to cultural activities',
       'Satisfaction with safety', 'Sense of a healthy environment',
       'Demographic', 'id'],
      dtype='object')

In [91]:
demographic_dict = {'Centr':'Central Okanagan',
                    'Central ':'Central Okanagan',
                    'North Okanaga':'North Okanagan',
                    'North Thompson':'North Okanagan',
                    'South Thompson':'South Okanagan'}

In [92]:
original_data = original_data.replace({'Demographic': demographic_dict})
original_data.groupby('Demographic')['Demographic'].count()

Demographic
Boundary Country                             8
Central Okanagan                           767
Gold Country                                 2
I do not live in the Thompson Okanagan.      2
North Okanagan                             127
Shuswap                                     21
Similkameen Valley                           5
South Okanagan                             115
Unknown                                      2
V0E 2W1                                      1
V4t1e8                                       1
Name: Demographic, dtype: int64

In [93]:
original_data.loc[~original_data['Demographic'].isin(['Boundary Country','Central Okanagan', 'Gold Country','North Okanagan', 'Shuswap',
                            'Similkameen Valley', 'South Okanagan']), 'Demographic'] = np.nan

In [94]:
original_data.head()

,Satisfaction with life,Sense of belonging in community,Satisfaction with tourism,Satisfaction with tourism .1,Tourism,Jobs with tourism,Entrepreneurship with tourism,Local culture from tourism,Production of local productions with tourism,Views on policies on tourism,Satisfaction with the quality of basic educational services,Satisfaction with the quality of basic healh services,Satisfaction with access to recreation,Satisfaction with access to cultural activities,Satisfaction with safety,Sense of a healthy environment,Demographic,id
0,80.0,75.0,70.0,Increase,NaN,NaN,100.0,100.0,100.0,100.0,100.0,NaN,75.0,100.0,50.0,75.0,Central Okanagan,0
1,60.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,50.0,100.0,75.0,NaN,1
2,80.0,100.0,NaN,Increase,NaN,NaN,100.0,100.0,100.0,100.0,100.0,NaN,100.0,0.0,100.0,75.0,Central Okanagan,2
3,70.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,75.0,75.0,50.0,NaN,3
4,70.0,100.0,90.0,Increase,Nearby BC communities | All of BC | Ot...,British Columbia Visitors | Canadian V...,75.0,75.0,75.0,75.0,75.0,Yes,75.0,75.0,50.0,50.0,Central Okanagan,4


In [95]:
original_data.groupby('Demographic')['Demographic'].count()

Demographic
Boundary Country        8
Central Okanagan      767
Gold Country            2
North Okanagan        127
Shuswap                21
Similkameen Valley      5
South Okanagan        115
Name: Demographic, dtype: int64

In [96]:
demographic_df = original_data[['id','Demographic']].copy()

## Indicator 1  
Sub-regional averages of `Satisfaction with life`

In [46]:
satisfaction_avg = pd.DataFrame(original_data.groupby('Demographic')['Satisfaction with life'].mean())
satisfaction_avg.reset_index(inplace=True)
satisfaction_avg.rename(columns={'Satisfaction with life':'value'}, inplace=True)
satisfaction_avg

,Demographic,value
0,Boundary Country,43.750000
1,Central Okanagan,60.573664
2,Gold Country,85.000000
3,North Okanagan,62.460317
4,Shuswap,57.619048
5,Similkameen Valley,74.000000
6,South Okanagan,65.130435


In [47]:
sat_avg_total = pd.DataFrame({'Demographic':['All Thompson Okanagan'], 'value':[original_data['Satisfaction with life'].mean()]})
sat_avg_total

,Demographic,value
0,All Thompson Okanagan,61.226054


In [48]:
satisfaction_avg = pd.concat([satisfaction_avg, sat_avg_total]).reset_index(drop=True)
satisfaction_avg

,Demographic,value
0,Boundary Country,43.750000
1,Central Okanagan,60.573664
2,Gold Country,85.000000
3,North Okanagan,62.460317
4,Shuswap,57.619048
5,Similkameen Valley,74.000000
6,South Okanagan,65.130435
7,All Thompson Okanagan,61.226054


In [49]:
indicator_1 = satisfaction_avg.rename(columns={'Demographic':'category_2'})
indicator_1['indicator'] = 'satisfaction_with_life'
indicator_1['region'] = 'Thompson Okanagan'
indicator_1['date'] = 2022
indicator_1['category_1'] = ''
indicator_1 = indicator_1[['indicator','region','date','category_1','category_2','value']]
indicator_1


,indicator,region,date,category_1,category_2,value
0,satisfaction_with_life,Thompson Okanagan,2022,,Boundary Country,43.750000
1,satisfaction_with_life,Thompson Okanagan,2022,,Central Okanagan,60.573664
2,satisfaction_with_life,Thompson Okanagan,2022,,Gold Country,85.000000
3,satisfaction_with_life,Thompson Okanagan,2022,,North Okanagan,62.460317
4,satisfaction_with_life,Thompson Okanagan,2022,,Shuswap,57.619048
5,satisfaction_with_life,Thompson Okanagan,2022,,Similkameen Valley,74.000000
6,satisfaction_with_life,Thompson Okanagan,2022,,South Okanagan,65.130435
7,satisfaction_with_life,Thompson Okanagan,2022,,All Thompson Okanagan,61.226054


## Transform to long format

In [97]:
df_long = pd.melt(original_data, id_vars=['id'], var_name='Topic', value_name='value')
df_long = df_long[~df_long['id'].isna()]
df_long

,id,Topic,value
0,0,Satisfaction with life,80.0
1,1,Satisfaction with life,60.0
2,2,Satisfaction with life,80.0
3,3,Satisfaction with life,70.0
4,4,Satisfaction with life,70.0
...,...,...,...
23217,1361,Demographic,Similkameen Valley
23218,1362,Demographic,NaN
23219,1363,Demographic,Central Okanagan
23220,1364,Demographic,North Okanagan


In [99]:
df_long = df_long[df_long['Topic'] != 'Demographic']

In [100]:
df_long_complete = pd.merge(df_long, demographic_df, on='id', how='left')
df_long_complete

,id,Topic,value,Demographic
0,0,Satisfaction with life,80.0,Central Okanagan
1,1,Satisfaction with life,60.0,NaN
2,2,Satisfaction with life,80.0,Central Okanagan
3,3,Satisfaction with life,70.0,NaN
4,4,Satisfaction with life,70.0,Central Okanagan
...,...,...,...,...
21851,1361,Sense of a healthy environment,75.0,Similkameen Valley
21852,1362,Sense of a healthy environment,NaN,NaN
21853,1363,Sense of a healthy environment,75.0,Central Okanagan
21854,1364,Sense of a healthy environment,50.0,North Okanagan


## 2) Modify / group data  
The following categories (questions/topics) need cleaning, separating multiple choice answers and/or removing invalid answers  
- Jobs with tourism  
- Tourism  
- Satisfaction with tourism .1  

The process will be to get the subset of data, modify it, and then remove and replace on the initial data table.

### Jobs with tourism 
**Question:**  
_If you saw an advertisement promoting your region as a place for tourists to visit would you agree it was appropriate for the following locations?  (Check all that apply)_

In [115]:
jobs = df_long_complete.loc[df_long_complete['Topic'] == 'Jobs with tourism'].copy()
jobs

,id,Topic,value,Demographic
6830,0,Jobs with tourism,NaN,Central Okanagan
6831,1,Jobs with tourism,NaN,NaN
6832,2,Jobs with tourism,NaN,Central Okanagan
6833,3,Jobs with tourism,NaN,NaN
6834,4,Jobs with tourism,British Columbia Visitors | Canadian V...,Central Okanagan
...,...,...,...,...
8191,1361,Jobs with tourism,Canadian Visitors | United States of America ...,Similkameen Valley
8192,1362,Jobs with tourism,NaN,NaN
8193,1363,Jobs with tourism,British Columbia Visitors | Canadian V...,Central Okanagan
8194,1364,Jobs with tourism,British Columbia Visitors,North Okanagan


In [116]:
jobs.value.unique()

array([nan,
       'British Columbia Visitors         | Canadian Visitors | United States of America        | Other countries',
       'British Columbia Visitors        ',
       'British Columbia Visitors         | Canadian Visitors | United States of America       ',
       'Canadian Visitors',
       'British Columbia Visitors         | Canadian Visitors | Other countries',
       'Other countries',
       'British Columbia Visitors         | Canadian Visitors',
       'United States of America       ',
       'British Columbia Visitors         | United States of America        | Other countries',
       'Canadian Visitors | United States of America       ',
       'Canadian Visitors | Other countries',
       'Canadian Visitors | United States of America        | Other countries',
       'British Columbia Visitors         | Other countries',
       'British Columbia Visitors         | United States of America       ',
       'United States of America        | Other countries'], dty

Get choices in different columns and then in long format

In [117]:
jobs_exp = jobs['value'].dropna().str.split('|', expand=True)
jobs_exp['id'] = jobs_exp.index
jobs_exp

,0,1,2,3,id
6834,British Columbia Visitors,Canadian Visitors,United States of America,Other countries,6834
6835,British Columbia Visitors,Canadian Visitors,United States of America,Other countries,6835
6836,British Columbia Visitors,Canadian Visitors,United States of America,Other countries,6836
6837,British Columbia Visitors,Canadian Visitors,United States of America,Other countries,6837
6838,British Columbia Visitors,Canadian Visitors,United States of America,Other countries,6838
...,...,...,...,...,...
8190,British Columbia Visitors,Canadian Visitors,United States of America,Other countries,8190
8191,Canadian Visitors,United States of America,Other countries,None,8191
8193,British Columbia Visitors,Canadian Visitors,None,None,8193
8194,British Columbia Visitors,None,None,None,8194


In [118]:
jobs_long = pd.melt(jobs_exp, id_vars=['id'], var_name='Topic', value_name='value')
jobs_long['Topic'] = 'Jobs with tourism'
jobs_long = jobs_long[~jobs_long['value'].isna()]
jobs_long['value'] = jobs_long['value'].str.strip()
jobs_long

,id,Topic,value
0,6834,Jobs with tourism,British Columbia Visitors
1,6835,Jobs with tourism,British Columbia Visitors
2,6836,Jobs with tourism,British Columbia Visitors
3,6837,Jobs with tourism,British Columbia Visitors
4,6838,Jobs with tourism,British Columbia Visitors
...,...,...,...
4097,8179,Jobs with tourism,Other countries
4100,8184,Jobs with tourism,Other countries
4101,8186,Jobs with tourism,Other countries
4102,8187,Jobs with tourism,Other countries


### Tourism  
**Question:**  
_I would welcome visitors from: (Check all that appy)_

Same process as before

In [108]:
tourism = df_long_complete.loc[df_long_complete['Topic'] == 'Tourism'].copy()
tourism_exp = tourism['value'].dropna().str.split('|', expand=True)
tourism_exp['id'] = tourism_exp.index
tourism_exp.head()



,0,1,2,3,4,id
5468,Nearby BC communities,All of BC,Other Canadian provinces,United States of America,Other countries,5468
5469,Nearby BC communities,All of BC,Other Canadian provinces,United States of America,Other countries,5469
5470,Nearby BC communities,All of BC,Other Canadian provinces,United States of America,Other countries,5470
5471,Nearby BC communities,All of BC,Other Canadian provinces,United States of America,Other countries,5471
5472,Nearby BC communities,All of BC,Other Canadian provinces,United States of America,Other countries,5472


In [109]:
tourism_long = pd.melt(tourism_exp, id_vars=['id'], var_name='Topic', value_name='value')
tourism_long['Topic'] = 'Tourism'
tourism_long = tourism_long[~tourism_long['value'].isna()]
tourism_long['value'] = tourism_long['value'].str.strip()
tourism_long

,id,Topic,value
0,5468,Tourism,Nearby BC communities
1,5469,Tourism,Nearby BC communities
2,5470,Tourism,Nearby BC communities
3,5471,Tourism,Nearby BC communities
4,5472,Tourism,Nearby BC communities
...,...,...,...
5189,6821,Tourism,Other countries
5190,6824,Tourism,Other countries
5192,6827,Tourism,Other countries
5193,6828,Tourism,Other countries


### Satisfaction with tourism .1

In [111]:
df_long_complete[df_long_complete['Topic'] == 'Satisfaction with tourism .1']['value'].unique()

array(['Increase ', nan, 'Stay the same ', 'I do not have an opinion.',
       'Decrease', '特に意見はない', 'Meningkat'], dtype=object)

In [112]:
satisfaction = df_long_complete.loc[(df_long_complete['Topic'] == 'Satisfaction with tourism .1') & (df_long_complete['value']\
    .isin(['Increase ', 'Stay the same ', 'I do not have an opinion.','Decrease']))].copy()
satisfaction

,id,Topic,value,Demographic
4098,0,Satisfaction with tourism .1,Increase,Central Okanagan
4100,2,Satisfaction with tourism .1,Increase,Central Okanagan
4102,4,Satisfaction with tourism .1,Increase,Central Okanagan
4103,5,Satisfaction with tourism .1,Stay the same,Central Okanagan
4105,7,Satisfaction with tourism .1,Increase,Central Okanagan
...,...,...,...,...
5458,1360,Satisfaction with tourism .1,Stay the same,North Okanagan
5459,1361,Satisfaction with tourism .1,Increase,Similkameen Valley
5461,1363,Satisfaction with tourism .1,Stay the same,Central Okanagan
5462,1364,Satisfaction with tourism .1,I do not have an opinion.,North Okanagan


### Combine with dataset  

In [123]:
df_long_updated = df_long_complete.loc[~(df_long_complete['Topic'].isin(['Satisfaction with tourism .1', 'Tourism', 'Jobs with tourism']))]
df_long_updated = df_long_updated.append([satisfaction, jobs_long, tourism_long])

In [124]:
df_long_updated[df_long_updated['Topic'] == 'Jobs with tourism']['value'].value_counts()

British Columbia Visitors    941
Canadian Visitors            858
Other countries              719
United States of America     678
Name: value, dtype: int64

In [129]:
df_long_updated['Topic'] = df_long_updated['Topic'].str.strip()
df_long_updated['Topic'] = df_long_updated['Topic'].str.lower().str.replace(' ', '_')
df_long_updated['region'] = 'Thompson Okanagan'
df_long_updated['date'] = 2022
df_long_updated['category_1'] = ''
df_long_updated.rename(columns={'Topic':'indicator', 'Demographic':'category_2'}, inplace=True)

In [130]:
df_long_updated

,id,indicator,value,category_2,region,date,category_1
0,0,satisfaction_with_life,80.0,Central Okanagan,Thompson Okanagan,2022,
1,1,satisfaction_with_life,60.0,NaN,Thompson Okanagan,2022,
2,2,satisfaction_with_life,80.0,Central Okanagan,Thompson Okanagan,2022,
3,3,satisfaction_with_life,70.0,NaN,Thompson Okanagan,2022,
4,4,satisfaction_with_life,70.0,Central Okanagan,Thompson Okanagan,2022,
...,...,...,...,...,...,...,...
5189,6821,tourism,Other countries,NaN,Thompson Okanagan,2022,
5190,6824,tourism,Other countries,NaN,Thompson Okanagan,2022,
5192,6827,tourism,Other countries,NaN,Thompson Okanagan,2022,
5193,6828,tourism,Other countries,NaN,Thompson Okanagan,2022,


In [131]:
indicator_csv = df_long_updated[['indicator','region','date','category_1','category_2','value']]
indicator_csv

,indicator,region,date,category_1,category_2,value
0,satisfaction_with_life,Thompson Okanagan,2022,,Central Okanagan,80.0
1,satisfaction_with_life,Thompson Okanagan,2022,,NaN,60.0
2,satisfaction_with_life,Thompson Okanagan,2022,,Central Okanagan,80.0
3,satisfaction_with_life,Thompson Okanagan,2022,,NaN,70.0
4,satisfaction_with_life,Thompson Okanagan,2022,,Central Okanagan,70.0
...,...,...,...,...,...,...
5189,tourism,Thompson Okanagan,2022,,NaN,Other countries
5190,tourism,Thompson Okanagan,2022,,NaN,Other countries
5192,tourism,Thompson Okanagan,2022,,NaN,Other countries
5193,tourism,Thompson Okanagan,2022,,NaN,Other countries


## 3) Add questions to dataset  
Load corresponding sheet from workbook, join by topic

In [45]:
sheet_name = 'questions_table'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

question_data = pd.read_csv(url)
question_data['Topic'] = question_data['Topic'].str.strip()
question_data


,Topic,Question
0,Satisfaction with life,"Overall, how satisfied are you with your life ..."
1,Sense of belonging in community,How would you describe your feeling of belongi...
2,Satisfaction with tourism,How satisfied are you with the state of touris...
3,Satisfaction with tourism,"Overall, the number of tourists to my site sho..."
4,Tourism,I would welcome visitors from: (Check all that...
5,Jobs with tourism,If you saw an advertisement promoting your reg...
6,Entrepreneurship with tourism,Tourism creates jobs for local people at my site.
7,Local culture from tourism,Tourism promotes local entrepreneurship at my ...
8,Production of local productions with tourism,Tourism promotes the local culture at my site.
9,Views on policies on tourism,Tourism promotes production of local products ...


In [47]:
df_long_updated.Topic.unique()

array(['Satisfaction with life', 'Sense of belonging in community',
       'Satisfaction with tourism', 'Entrepreneurship with tourism',
       'Local culture from tourism',
       'Production of local productions with tourism',
       'Views on policies on tourism',
       'Satisfaction with the quality of basic educational services',
       'Satisfaction with the quality of basic healh services',
       'Satisfaction with access to recreation',
       'Satisfaction with access to cultural activities',
       'Satisfaction with safety', 'Sense of a healthy environment',
       'Satisfaction with tourism .1', 'Jobs with tourism', 'Tourism'],
      dtype=object)

In [48]:
question_data.iloc[3]['Topic'] = 'Satisfaction with tourism .1'
question_data

,Topic,Question
0,Satisfaction with life,"Overall, how satisfied are you with your life ..."
1,Sense of belonging in community,How would you describe your feeling of belongi...
2,Satisfaction with tourism,How satisfied are you with the state of touris...
3,Satisfaction with tourism .1,"Overall, the number of tourists to my site sho..."
4,Tourism,I would welcome visitors from: (Check all that...
5,Jobs with tourism,If you saw an advertisement promoting your reg...
6,Entrepreneurship with tourism,Tourism creates jobs for local people at my site.
7,Local culture from tourism,Tourism promotes local entrepreneurship at my ...
8,Production of local productions with tourism,Tourism promotes the local culture at my site.
9,Views on policies on tourism,Tourism promotes production of local products ...


In [49]:
df_long = pd.merge(df_long_updated, question_data, on='Topic', how='left')
df_long

,id,Topic,Answer,Question
0,0,Satisfaction with life,80,"Overall, how satisfied are you with your life ..."
1,1,Satisfaction with life,60,"Overall, how satisfied are you with your life ..."
2,2,Satisfaction with life,80,"Overall, how satisfied are you with your life ..."
3,3,Satisfaction with life,70,"Overall, how satisfied are you with your life ..."
4,4,Satisfaction with life,70,"Overall, how satisfied are you with your life ..."
...,...,...,...,...
26051,6816,Tourism,Other countries,I would welcome visitors from: (Check all that...
26052,6819,Tourism,Other countries,I would welcome visitors from: (Check all that...
26053,6822,Tourism,Other countries,I would welcome visitors from: (Check all that...
26054,6823,Tourism,Other countries,I would welcome visitors from: (Check all that...


## 4) Save data  
The dataset is ready for use in visualizations (Tableau or Python) or for further formatting for the API when the data model is ready

In [50]:
df_long.to_csv('../data/local_satisfaction_long_data.csv', index=False)